Problem Set 3: Ulaanbaatar Air Pollution

Chinguun Batsaikhan


1) Importing Data

In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import *
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [240]:
X = pd.read_csv('/content/drive/MyDrive/pset3_data_fixed/X.csv', index_col='id')

In [241]:
y = pd.read_csv('/content/drive/MyDrive/pset3_data_fixed/y.csv', index_col='id')

In [242]:
weather = pd.read_csv('/content/drive/MyDrive/pset3_data_fixed/weather.csv')

In [243]:
test = pd.read_csv('/content/drive/MyDrive/pset3_data_fixed/test.csv', index_col='id')

2) Data Cleaning

In [244]:
X.head()

,type,source,unit,date,value1,value2,value3
id,,,,,,,
0,PM2.5,Stateair.mn,µg/m3,2015-12-12 06:00:00,NaN,NaN,NaN
1,PM2.5,Stateair.mn,µg/m3,2015-12-12 07:00:00,242.0,NaN,NaN
2,PM2.5,Stateair.mn,µg/m3,2015-12-12 08:00:00,244.0,242.0,NaN
3,PM2.5,Stateair.mn,µg/m3,2015-12-12 09:00:00,93.0,244.0,242.0
4,PM2.5,Stateair.mn,µg/m3,2015-12-12 10:00:00,53.0,93.0,244.0


In [245]:
X.dtypes

type       object
source     object
unit       object
date       object
value1    float64
value2    float64
value3    float64
dtype: object

In [246]:
X.type.unique()

array(['PM2.5'], dtype=object)

In [247]:
X = X.drop(columns=['type', 'source', 'unit', 'date', 'value2', 'value3'])

In [248]:
test = test.drop(columns=['type', 'source', 'unit', 'date', 'value2', 'value3'])

* Dropped value2 and value3 because they repeat and in the end gives back to many NaN values

In [249]:
X['value1'] = X['value1'].replace(-999.0, np.nan)

In [250]:
X = X.fillna(value=X.median())

In [251]:
X.isnull().sum()

value1    0
dtype: int64

In [252]:
test['value1'] = test['value1'].replace(-999, np.nan)

In [253]:
test = test.fillna(value=test.median())

In [254]:
test.isnull().sum()

value1    0
dtype: int64

In [255]:
test.head()

,value1
id,
21158,56.0
21159,23.0
21160,20.0
21161,17.0
21162,5.0


In [256]:
weather.head()

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windGustTime,windBearing,cloudCover,uvIndex,visibility,ozone,precipType,precipAccumulation
0,1.449850e+09,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.99,0.75,NaN,0.0,NaN,NaN,NaN,0.44,0.0,10.003,NaN,NaN,NaN
1,1.449853e+09,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,NaN,1.0,NaN,NaN,NaN,0.44,0.0,10.003,NaN,NaN,NaN
2,1.449857e+09,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.65,-19.65,-21.96,0.82,1041.1,0.0,NaN,NaN,NaN,0.35,0.0,10.003,NaN,NaN,NaN
3,1.449860e+09,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,NaN,0.0,NaN,NaN,NaN,0.44,0.0,8.005,NaN,NaN,NaN
4,1.449864e+09,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,NaN,0.0,NaN,NaN,NaN,0.44,0.0,6.004,NaN,NaN,NaN


 Dropping Columns

In [257]:
weather = weather.drop(columns=['time', 'windGustTime', 'ozone', 'precipType', 'precipAccumulation'])

In [258]:
weather[weather.select_dtypes(['object']).columns] = weather.select_dtypes(['object']).fillna('null')

In [259]:
weather = weather.fillna(value=weather.median())

Merging dataframes into one data frame

In [260]:
X = pd.merge(left=X, right=weather, left_on=X.index, right_on=weather.index)

In [261]:
test = pd.merge(left=test, right=weather, left_on=test.index, right_on=weather.index)

In [262]:
X.head()

,key_0,value1,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility
0,0,23.0,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.99,0.75,1023.4,0.0,3.56,221.0,0.44,0.0,10.003
1,1,242.0,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,1023.4,1.0,3.56,221.0,0.44,0.0,10.003
2,2,244.0,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.65,-19.65,-21.96,0.82,1041.1,0.0,3.56,221.0,0.35,0.0,10.003
3,3,93.0,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,1023.4,0.0,3.56,221.0,0.44,0.0,8.005
4,4,53.0,Partly Cloudy,partly-cloudy-night,0.0,0.0,-19.72,-19.72,-22.01,0.82,1023.4,0.0,3.56,221.0,0.44,0.0,6.004


One Hot

In [263]:
enc = OneHotEncoder(handle_unknown='ignore')

In [264]:
X_cat_enc = pd.DataFrame(enc.fit_transform(X.select_dtypes('object')).toarray())

In [265]:
test_cat_enc = pd.DataFrame(enc.transform(test.select_dtypes('object')).toarray())

In [266]:
X = X.drop(columns=['summary', 'icon']).join(X_cat_enc)

In [267]:
test = test.drop(columns=['summary', 'icon']).join(test_cat_enc)

In [268]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.3, random_state=40)

Splitting Data

In [269]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.3, random_state=40)

Random Forest Regressor

In [270]:
rf = ensemble.RandomForestRegressor(max_features=7, n_estimators=200, n_jobs=-1, random_state=40)

In [271]:
rf.fit(X_train, np.ravel(y_train))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=False,
                      random_state=40, verbose=0, warm_start=False)

In [272]:
rf_pred = rf.predict(X_test)

In [273]:
np.sqrt(metrics.mean_squared_error(y_test, rf_pred))

198.86024026823773

Making Predictions

In [274]:
prediction = rf.predict(test)

In [275]:
prediction = pd.DataFrame(prediction, index=test.key_0, columns=['value'])

In [276]:
prediction.to_csv('prediction.csv', index=True)

In [277]:
prediction

,value
key_0,
21158,24.1050
21159,-349.8200
21160,11.3600
21161,23.0800
21162,-27.0500
...,...
25689,-106.5510
25690,-134.0990
25691,-101.9950
